# NB

## Load and import data

In [16]:
from src.data.make_dataset import main as get_data
from src.data.preprocess import Preprocessor

import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score,precision_score, recall_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, f1_score
from sklearn import model_selection, naive_bayes
import scipy
import matplotlib.pyplot as plt
import pickle
import joblib
import os

In [17]:
raw_reviews = pd.read_csv("../../data/raw/reviews.csv")
raw_reviews.head()

,Sentiment,Time,Text
0,positive,18/6/21,This is a very healthy dog food. Good for thei...
1,positive,7/7/21,I've been very pleased with the Natural Balanc...
2,positive,18/6/21,"Before I was educated about feline nutrition, ..."
3,positive,7/7/21,"My holistic vet recommended this, along with a..."
4,positive,1/7/21,I bought this coffee because its much cheaper ...


In [18]:
X_train, X_test, y_train, y_test = get_data(raw_reviews)

[PP] Preprocessing complete
[FE] finished lowercase count...
[FE] finished uppercase count...
[FE] finished uppercase ratio...
[FE] finished punc count...
[FE] finished pos tags...
[FE] finished pos tag count...
[FE] finished tokenized_untokenized count...
[FE] finished num words misspelled...
[FE] finished polarity...
[FE] finished subjectivity...


## Prepare data for NB
add normalisation of FE features to remove negative values (NB model does not accept this)

In [ ]:
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_clean_scaled = scaler.fit_transform(X_train_clean)
X_train_clean_scaled = pd.DataFrame(X_train_clean_scaled, columns = X_train_clean.columns)
X_test_clean_scaled = scaler.fit_transform(X_test_clean)
X_test_clean_scaled = pd.DataFrame(X_test_clean_scaled, columns = X_train_clean.columns)

X_train_concat = pd.concat([X_train_clean_scaled, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean_scaled, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

In [ ]:
X_train_concat.shape #9213 features

In [ ]:
y_train

# Vanilla NB

In [ ]:
# fit the training dataset on the classifier
nb = naive_bayes.MultinomialNB()
nb.fit(X_train_concat,y_train)

### Evaluation

In [ ]:
# Make predictions on the testing set
y_pred = nb.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc = roc_auc_score(y_test, y_pred)
print(f"For this NB model, the AUC score is: {auc}")
#For this NB model, the AUC score is: 0.5294149882945051

## Set max_features=5000 for TfidfVectorizer & refit into Vanilla NB

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_clean_scaled = scaler.fit_transform(X_train_clean)
X_train_clean_scaled = pd.DataFrame(X_train_clean_scaled, columns = X_train_clean.columns)
X_test_clean_scaled = scaler.fit_transform(X_test_clean)
X_test_clean_scaled = pd.DataFrame(X_test_clean_scaled, columns = X_train_clean.columns)

X_train_concat = pd.concat([X_train_clean_scaled, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean_scaled, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

# NB with max_features (5000)

In [ ]:
# fit the training dataset on the classifier
nb = naive_bayes.MultinomialNB()
nb.fit(X_train_concat,y_train)

### Evaluation

In [ ]:
# Make predictions on the testing set
y_pred = nb.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1_score)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report) #better

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc = roc_auc_score(y_test, y_pred)
print(f"For this NB model, the AUC score is: {auc}")
#For this NB model, the AUC score is: 0.6159874264570492

## Set max_features=1000 for TfidfVectorizer & refit into Vanilla NB

In [ ]:
vectorizer = TfidfVectorizer(max_features=1000)

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_clean_scaled = scaler.fit_transform(X_train_clean)
X_train_clean_scaled = pd.DataFrame(X_train_clean_scaled, columns = X_train_clean.columns)
X_test_clean_scaled = scaler.fit_transform(X_test_clean)
X_test_clean_scaled = pd.DataFrame(X_test_clean_scaled, columns = X_train_clean.columns)

X_train_concat = pd.concat([X_train_clean_scaled, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean_scaled, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

# NB with max_features (1000)

In [ ]:
# fit the training dataset on the classifier
nb = naive_bayes.MultinomialNB()
nb.fit(X_train_concat,y_train)

### Evaluation

In [ ]:
# Make predictions on the testing set
y_pred = nb.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report) #better

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc = roc_auc_score(y_test, y_pred)
print(f"For this NB model, the AUC score is: {auc}")

# NB with dim reduction (SVD: 100 components)

In [ ]:
n_components = 100
svd = TruncatedSVD(n_components=n_components)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)

In [ ]:
X_train_svd_df = pd.DataFrame(X_train_svd, columns=[f'svd_{i}' for i in range(n_components)])
X_test_svd_df = pd.DataFrame(X_test_svd, columns=[f'svd_{i}' for i in range(n_components)])

X_train_concat = pd.concat([X_train_clean, X_train_svd_df], axis=1)
X_test_concat = pd.concat([X_test_clean, X_test_svd_df], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_concat_svd = scaler.fit_transform(X_train_concat)
X_train_concat_svd = pd.DataFrame(X_train_concat_svd, columns = X_train_concat.columns)
X_test_concat_svd = scaler.fit_transform(X_test_concat)
X_test_concat_svd = pd.DataFrame(X_test_concat_svd, columns = X_test_concat.columns)

In [ ]:
# fit the training dataset on the classifier
nb = naive_bayes.MultinomialNB()
nb.fit(X_train_concat_svd,y_train)

In [ ]:
# Make predictions on the testing set
y_pred = nb.predict(X_test_concat_svd)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1_score)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

# NB with oversampling of minority class

In [ ]:
X_train_os, X_test_os, y_train_os, y_test_os = get_data("../data-dialogue/data/raw/reviews.csv", oversample=True)

In [ ]:
#max features = 1000 was the best model thus far
vectorizer = TfidfVectorizer(max_features=1000)

X_train_tfidf = vectorizer.fit_transform(X_train_os['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test_os['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train_os.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test_os.drop(['cleaned_text', 'text'], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_clean_scaled = scaler.fit_transform(X_train_clean)
X_train_clean_scaled = pd.DataFrame(X_train_clean_scaled, columns = X_train_clean.columns)
X_test_clean_scaled = scaler.fit_transform(X_test_clean)
X_test_clean_scaled = pd.DataFrame(X_test_clean_scaled, columns = X_train_clean.columns)

X_train_concat = pd.concat([X_train_clean_scaled, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean_scaled, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

In [ ]:
# fit the training dataset on the classifier
nb = naive_bayes.MultinomialNB()
nb.fit(X_train_concat,y_train_os)

In [ ]:
# Make predictions on the testing set
y_pred = nb.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test_os, y_pred)
precision = precision_score(y_test_os,y_pred)
recall = recall_score(y_test_os, y_pred)
conf_matrix = confusion_matrix(y_test_os, y_pred)
class_report = classification_report(y_test_os, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

# Using ComplemenNB instead of MultinomialNB
- Suited from imbalanced data sets

In [ ]:
# fit the training dataset on the classifier
nb = naive_bayes.ComplementNB()
nb.fit(X_train_concat,y_train_os)

In [ ]:
# Make predictions on the testing set
y_pred = nb.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test_os, y_pred)
precision = precision_score(y_test_os,y_pred) #performing very badly
recall = recall_score(y_test_os, y_pred)
conf_matrix = confusion_matrix(y_test_os, y_pred)
class_report = classification_report(y_test_os, y_pred)
f1 = f1_score(y_test_os, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

# Evaluation:
best model = tfidf with max_features=1000 + oversampling of minority class

In [ ]:
#max features = 1000 was the best model thus far
vectorizer = TfidfVectorizer(max_features=1000)

X_train_tfidf = vectorizer.fit_transform(X_train_os['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test_os['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train_os.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test_os.drop(['cleaned_text', 'text'], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_clean_scaled = scaler.fit_transform(X_train_clean)
X_train_clean_scaled = pd.DataFrame(X_train_clean_scaled, columns = X_train_clean.columns)
X_test_clean_scaled = scaler.fit_transform(X_test_clean)
X_test_clean_scaled = pd.DataFrame(X_test_clean_scaled, columns = X_test_clean.columns)

X_train_concat = pd.concat([X_train_clean_scaled, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean_scaled, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

# Parameter Tuning

In [ ]:
cv_method = RepeatedStratifiedKFold(n_splits=3, 
                                    n_repeats=1, 
                                    random_state=999)

from sklearn.preprocessing import PowerTransformer

alphas = [0.1, 0.3, 0.5, 0.7, 1.0]
params_NB = {'alpha': alphas, 'fit_prior' : [True, False], 'class_prior' : [None, [.1,.9],[.2, .8]]}

gs_NB = GridSearchCV(estimator=nb, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy')

gs_NB.fit(X_train_concat, y_train_os)

In [ ]:
gs_NB.best_params_ #{'alpha': 0.5, 'class_prior': None, 'fit_prior': True}

In [ ]:
grid_predictions = gs_NB.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, grid_predictions)
precision = precision_score(y_test,grid_predictions)
recall = recall_score(y_test, grid_predictions)
conf_matrix = confusion_matrix(y_test, grid_predictions)
class_report = classification_report(y_test, grid_predictions)
f1 = f1_score(y_test, grid_predictions)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

In [ ]:
gs_NB.best_estimator_ 
# MultinomialNB(alpha=0.5)

In [ ]:
# save best parameters
joblib.dump(gs_NB.best_estimator_, 'naive_bayes_best_model.pkl')

# Load Saved Model

In [ ]:
saved_model = joblib.load('naive_bayes_best_model.pkl')